<a href="https://colab.research.google.com/github/dhrubjun/Neural_Network/blob/main/mnist_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from keras.callbacks import ModelCheckpoint , EarlyStopping

In [ ]:
!nvidia-smi

Tue Jul 19 11:05:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/data/digit-recognizer/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/data/digit-recognizer/test.csv')

In [ ]:
df_train.shape, df_test.shape

((42000, 785), (28000, 784))

In [ ]:
df_train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
pd.DataFrame(df_train.label.value_counts())

,label
1,4684
7,4401
3,4351
9,4188
2,4177
6,4137
0,4132
4,4072
8,4063
5,3795


In [ ]:
df_train.label.value_counts().sum()

42000

In [ ]:
df_train.isna().sum()

label       0
pixel0      0
pixel1      0
pixel2      0
pixel3      0
           ..
pixel779    0
pixel780    0
pixel781    0
pixel782    0
pixel783    0
Length: 785, dtype: int64

In [ ]:
sum(df_train.isna().sum())

0

In [ ]:
df_train['pixel0'].max(), df_train['pixel0'].min(),

(0, 0)

In [ ]:
X = df_train.drop('label', axis=1)
y = df_train['label']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)


In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((33600, 784), (8400, 784), (33600,), (8400,))

In [ ]:
from tensorflow.python.ops.gen_array_ops import expand_dims
model = tf.keras.Sequential([
        tf.keras.layers.Dense(350, input_shape=(784,),  activation='relu'),
        tf.keras.layers.Dense(200, activation='relu'),
        tf.keras.layers.Dense(100, activation='relu'),
        tf.keras.layers.Dense(50, activation='relu'),
        tf.keras.layers.Dense(20, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')])

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer = tf.keras.optimizers.Adam(),
              metrics = ["accuracy"])




In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 350)               274750    
                                                                 
 dense_1 (Dense)             (None, 200)               70200     
                                                                 
 dense_2 (Dense)             (None, 100)               20100     
                                                                 
 dense_3 (Dense)             (None, 50)                5050      
                                                                 
 dense_4 (Dense)             (None, 20)                1020      
                                                                 
 dense_5 (Dense)             (None, 10)                210       
                                                                 
Total params: 371,330
Trainable params: 371,330
Non-trai

# Creating a checkpoint to save the model weights that lead to the highest validation accuracy

In [ ]:


checkpoint = ModelCheckpoint('weights.hdf5' , monitor = 'val_accuracy' , save_best_only = True )

# Forcing the model to stop when 5 epochs have passed without increasing the validation accuracy value

In [ ]:
early_stopping = EarlyStopping(monitor = 'val_accuracy' , patience = 5)

#Training the model

In [ ]:
history = model.fit(tf.expand_dims(X_train, axis=-1), y_train, epochs=20, validation_data=(X_test, y_test), callbacks = [checkpoint, early_stopping])

Epoch 1/20
1050/1050 [==============================] - 6s 3ms/step - loss: 2.1466 - accuracy: 0.2357 - val_loss: 1.6854 - val_accuracy: 0.3652
Epoch 2/20
1050/1050 [==============================] - 3s 3ms/step - loss: 1.2925 - accuracy: 0.5192 - val_loss: 0.7683 - val_accuracy: 0.7757
Epoch 3/20
1050/1050 [==============================] - 3s 3ms/step - loss: 0.5014 - accuracy: 0.8740 - val_loss: 0.2723 - val_accuracy: 0.9287
Epoch 4/20
1050/1050 [==============================] - 3s 3ms/step - loss: 0.2176 - accuracy: 0.9458 - val_loss: 0.2092 - val_accuracy: 0.9490
Epoch 5/20
1050/1050 [==============================] - 4s 3ms/step - loss: 0.1574 - accuracy: 0.9589 - val_loss: 0.1922 - val_accuracy: 0.9500
Epoch 6/20
1050/1050 [==============================] - 3s 3ms/step - loss: 0.1214 - accuracy: 0.9682 - val_loss: 0.1514 - val_accuracy: 0.9625
Epoch 7/20
1050/1050 [==============================] - 3s 3ms/step - loss: 0.1068 - accuracy: 0.9725 - val_loss: 0.1801 - val_accuracy:

In [ ]:
pd.DataFrame(history.history)

,loss,accuracy,val_loss,val_accuracy
0,2.146603,0.235655,1.685362,0.365238
1,1.292530,0.519226,0.768252,0.775714
2,0.501406,0.873988,0.272267,0.928690
3,0.217632,0.945774,0.209160,0.949048
4,0.157413,0.958899,0.192190,0.950000
5,0.121371,0.968185,0.151353,0.962500
6,0.106772,0.972500,0.180129,0.959643
7,0.093716,0.974940,0.184149,0.957857
8,0.083403,0.977500,0.172619,0.961071
9,0.070247,0.981577,0.204468,0.958809


In [ ]:
max(pd.DataFrame(history.history).val_accuracy)

0.9624999761581421

In [ ]:
print("No. of Epochs :", len(pd.DataFrame(history.history).val_accuracy))

No. of Epochs : 11


In [ ]:
predictions = model.predict(df_test).argmax(axis=1)

In [ ]:
predictions.shape

(28000,)

In [ ]:
df_test['Label'] = predictions
df_test['ImageId'] = list(range(1, 28001 , 1))

In [ ]:
df_test[['ImageId' , 'Label']].to_csv('submission.csv' , index = False)

In [ ]:
from google.colab import files

In [ ]:
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>